In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive

In [0]:
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive/Colab Notebooks/text-summarization")

1. 数据太少，只有7000多
2. max_length 太长
3. 训练方式，随机训练

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
SOS_token = 0
EOS_token = 1
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs(train_x, train_y):
    print("Reading lines...")

    # Read the file and split into lines
    lines_x = open('%s' % (train_x), encoding='utf-8'). \
        read().strip().split('\n')
    lines_y = open('%s' % (train_y), encoding='utf-8'). \
        read().strip().split('\n')

    lines = lines_x
    for i in range(len(lines_y)):
        lines[i] = lines[i]+"\t"+lines_y[i]


    # get pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    lang = Lang('english')

    return lang, pairs

In [0]:
MAX_LENGTH = 40
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(train_x, train_y):
    lang, pairs = readLangs(train_x, train_y)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        lang.addSentence(pair[0])
        lang.addSentence(pair[1])
    print("Counted words:")
    print(lang.name, lang.n_words)
    return lang, pairs


lang, pairs = prepareData("data/train/article_train.txt", "data/train/title_train.txt")
print(random.choice(pairs))

Reading lines...
Read 8000 sentence pairs
Trimmed to 7052 sentence pairs
Counting words...
Counted words:
english 18614
['an israeli soldier and hezbollah fighter were killed in clashes friday in south lebanon that sparked an israeli air strike and an artillery duel security sources in the region said .', 'israeli soldier hezbollah fighter killed in south lebanon']


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
      

In [0]:
def indexesFromSentence(lang, sentence):
    res = list()
    for word in sentence.split(' '):
        if word in lang.word2index:
            res.append(lang.word2index[word])
        else:
            res.append(2)
    return res
#     return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0])
    target_tensor = tensorFromSentence(lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
# teacher_forcing_ratio = 0.1


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

# #     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing

#     else:
#         # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break
    
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
plot_losses = []

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
#         print("loss: ", loss)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            evaluateRandomly(encoder, decoder, 1)

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [21]:
hidden_size = 256
encoder1 = EncoderRNN(lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, len(pairs), print_every=100)

loss:  9.841120402018229
loss:  9.816164016723633
loss:  9.850018183390299
loss:  9.791709899902344
loss:  9.777386665344238
loss:  9.82783711751302
loss:  5.447140163845486
loss:  5.361182126131925
loss:  2.9660964965820313
loss:  3.246868133544922
loss:  2.9403913497924803
loss:  1.960769271850586
loss:  5.849392700195312
loss:  8.129722595214844
loss:  9.748071943010602
loss:  1.9770782470703125
loss:  3.2538028293185763
loss:  9.73447036743164
loss:  3.2555211385091147
loss:  1.9623571395874024
loss:  3.1998939514160156
loss:  2.969731330871582
loss:  3.2642845577663846
loss:  3.2899487813313804
loss:  3.6819517612457275
loss:  4.214836665562221
loss:  2.639025254683061
loss:  5.958208084106445
loss:  1.3212042490641276
loss:  9.76559787326389
loss:  5.887322998046875
loss:  3.2744907803005643
loss:  3.2503142886691623
loss:  4.212016241891043
loss:  2.7971624646868025
loss:  3.661893367767334
loss:  3.2809545728895397
loss:  9.703243255615234
loss:  2.479511260986328
loss:  3.2730

In [22]:
plot_losses

[4.422941173130178,
 3.500967760758347,
 3.2238949595311204,
 4.6518692631822605,
 5.008647417226477,
 5.002466449474809,
 5.808886403697674,
 5.329220066403164,
 5.673936981081706,
 5.424304878304685,
 5.81765570044803,
 5.517854626853893,
 5.5267296025708825,
 5.570897422970551,
 5.439702729057164,
 5.49813761284616,
 5.521222512628017,
 5.680324592736098,
 5.658935553454284,
 5.502003616795491,
 5.678249212926997,
 5.507017130225489,
 5.856929863412578,
 5.402856086273316,
 5.4363760081510994,
 5.37328777362587,
 5.420704831955445,
 5.480795089580584,
 5.415265697522684,
 5.6746369549838285,
 5.7802357124353385,
 5.245638479447255,
 5.642440013958858,
 6.0877442822890915,
 5.41067464704392,
 5.407706816874244,
 5.344468341013912,
 5.639571918530874,
 5.350855019122252,
 5.558348032448544,
 5.6314723088540735,
 5.531407517348056,
 4.971793151263812,
 5.3956675989582585,
 5.174037759302617,
 5.561995922860693,
 5.254962729387623,
 5.572562444940843,
 5.596815524490974,
 5.506974199568

In [29]:
_, pairs_test = prepareData("data/test/article_test.txt", "data/test/title_test.txt")


def cal_ROUGE_testset(pairs_test, print_every=200):
    rouge1 = 0
    rouge2 = 0
    for i in range(len(pairs_test)):
        predict_output, _ = evaluate(encoder1, attn_decoder1, pairs_test[i][0])
        predict_output = ' '.join(predict_output)
        rouge1 += Rouge_1(predict_output, pairs_test[i][1])
        rouge2 += Rouge_2(predict_output, pairs_test[i][1])
        if i % print_every == 0:
            print(pairs_test[i][0] + "  ----->  " + predict_output)

    return rouge1 / len(pairs_test), rouge2 / len(pairs_test)


import jieba


# 使用jieba进行分词
def Rouge_1(model, reference):  # terms_reference为参考摘要，terms_model为候选摘要   ***one-gram*** 一元模型
    terms_reference = jieba.cut(reference)  # 默认精准模式
    terms_model = jieba.cut(model)
    grams_reference = list(terms_reference)
    grams_model = list(terms_model)
    grams_reference = [x for x in grams_reference if x != " "]
    grams_model = [x for x in grams_model if x != " "]
    temp = 0
    ngram_all = len(grams_reference)
    for x in grams_reference:
        if x in grams_model: temp = temp + 1
    rouge_1 = temp / ngram_all
    return rouge_1


def Rouge_2(model, reference):  # terms_reference为参考摘要，terms_model为候选摘要   ***Bi-gram***  2元模型
    terms_reference = jieba.cut(reference)
    terms_model = jieba.cut(model)
    grams_reference = list(terms_reference)
    grams_model = list(terms_model)
    grams_reference = [x for x in grams_reference if x != " "]
    grams_model = [x for x in grams_model if x != " "]
    gram_2_model = []
    gram_2_reference = []
    temp = 0
    ngram_all = len(grams_reference) - 1
    for x in range(len(grams_model) - 1):
        gram_2_model.append(grams_model[x] + grams_model[x + 1])
    for x in range(len(grams_reference) - 1):
        gram_2_reference.append(grams_reference[x] + grams_reference[x + 1])
    for x in gram_2_model:
        if x in gram_2_reference: temp = temp + 1
    rouge_2 = temp / ngram_all
    return rouge_2


ROUGE1, ROUGE2 = cal_ROUGE_testset(pairs_test)

print("testset: ROUGE1 ->", ROUGE1)
print("testset: ROUGE2 ->", ROUGE2)

Reading lines...
Read 2000 sentence pairs
Trimmed to 1781 sentence pairs
Counting words...
Counted words:
english 9256
leaders from central and eastern europe met on monday to discuss ways of improving the living conditions of gypsies in the european union lrb eu rrb candidate countries .  ----->  chinese to to in to <EOS>
china s ministry of construction is to host a major international building technology and equipment exhibition focusing on the hi tech sector in beijing from july to .  ----->  chinese to to on in <EOS>
royal caribbean cruises said wednesday it took a loss in the second quarter as people booked fewer cruises amid concerns over swine flu and the recession .  ----->  chinese to to in to <EOS>
king hussein of jordan will meet with president bill clinton in washington on wednesday the white house said .  ----->  two to in to in to <EOS>
cote d ivoire rebels clashed with french troops in the western town of duekoue on friday but there were no deaths on the french side a f